In [1]:
import pickle
import json
from pprint import pprint 
import numpy as np

from collections import Counter
from copy import deepcopy

# load data

### adjectives

In [2]:
adjectives = pickle.load( open( "pkl/adjectives_sorted.pkl", "rb" ) )

FileNotFoundError: [Errno 2] No such file or directory: 'pkl/adjectives_sorted.pkl'

### all adjectives

In [ ]:
all_adjectives = pickle.load( open( "pkl/all_adjectives.pkl", "rb" ) )

### artists

In [5]:
artists = pickle.load( open( "pkl/artists_final.pkl", "rb" ) )

### dj artists

In [19]:
dj_artists = pickle.load( open( "pkl/dj-artists_final.pkl", "rb" ) )

### djs

In [17]:
djs = pickle.load( open( "pkl/djs_final.pkl", "rb" ) )

### event adjectives

In [22]:
event_adjectives = pickle.load( open( "pkl/event-adjectives_final.pkl", "rb" ) )

### event djs

In [24]:
event_djs = pickle.load( open( "pkl/event-djs_final.pkl", "rb" ) )

### events

In [26]:
events = pickle.load( open( "pkl/events_final.pkl", "rb" ) )

### labels

In [6]:
labels = pickle.load( open( "pkl/labels_final.pkl", "rb" ) )

### records

In [7]:
records = pickle.load( open( "pkl/records_final.pkl", "rb" ) )

In [32]:
records[100025]

{'record_styles': ['Progressive House', 'House'],
 'record_labels': ['9777-Ouchi'],
 'record_rating': 4.2,
 'record_artists': ['80903-Lost-In-Time'],
 'record_name': 'Workout',
 'record_year': '1994',
 'record_youtube_ids': None,
 'record_id': 100025,
 'countries': ['UK'],
 'record_genres': ['Electronic']}

# limit data to records with youtube_ids and rating over 4.4

In [44]:
len(records)

2730040

### filter records

In [68]:
filtered_records = {}
filtered_label_ids = []
filtered_artist_ids = []

count = 0

for record_id, record in records.items():
    if record['record_rating']:
        if record['record_youtube_ids'] \
        and record['record_rating'] > 4.4:
            filtered_records[record_id] = record
            
            filtered_label_ids.extend(record['record_labels'])
            filtered_artist_ids.extend(record['record_artists'])
            
filtered_label_ids = list(set(filtered_label_ids))
filtered_artist_ids = list(set(filtered_artist_ids))

### filter labels and artists

In [70]:
filtered_labels = {}
filtered_artists = {}

rejected_label_ids = []
rejected_artist_ids = []

for label_id in filtered_label_ids:
    try:
        filtered_labels[label_id] = labels[label_id]
    except:
        rejected_label_ids.append(label_id)
    
for artist_id in filtered_artist_ids:
    try:
        filtered_artists[artist_id] = artists[artist_id]
    except:
        rejected_artist_ids.append(artist_id)

### delete records with restricted artists or labels

In [75]:
out = {}
for record_id, record in filtered_records.items():
    pass_flag = True
    
    for artist in record['record_artists']:
        if artist in rejected_artist_ids:
            pass_flag = False
            
    for label in record['record_labels']:
        if label in rejected_label_ids:
            pass_flag = False
            
    if pass_flag:
        out[record_id] = record

filtered_records = out

### create filtered styles

In [83]:
filtered_styles = []

for record in filtered_records.values():
    filtered_styles.extend(record['record_styles'])

filtered_styles = list(set(filtered_styles))

In [81]:
len(filtered_labels)

83835

# dump json

### records

In [77]:
out_file = open("json/filtered_records.json", "w")
json.dump(filtered_records, out_file)
out_file.close()

### labels

In [78]:
out_file = open("json/filtered_labels.json", "w")
json.dump(filtered_labels, out_file)
out_file.close()

### artists

In [79]:
out_file = open("json/filtered_artists.json", "w")
json.dump(filtered_artists, out_file)
out_file.close()

### styles

In [87]:
out_file = open("json/filtered_styles.json", "w")
json.dump(filtered_styles, out_file)
out_file.close()

# create data

### event_adjecties -> label_adjectives

In [33]:
event_artists = {}
for event_id, djs in event_djs.items():
    event_artists[event_id] = []
    for dj in djs:
        if dj in dj_artists:
            temp_artists = dj_artists[dj]
            event_artists[event_id].extend(temp_artists)
    if event_artists[event_id] == []:
        event_artists.pop(event_id)

In [34]:
artist_labels = {}
for record_id, data in records.items():
    for artist_id in data['record_artists']:
        artist_labels[artist_id] = artist_labels.get(artist_id,[])
        artist_labels[artist_id].extend(data['record_labels'])
        
for artist_id, labels in artist_labels.items():
    labels = list(set(labels))
    artist_labels[artist_id] = labels

In [35]:
label_events = {}
for event_id, artist_ids in event_artists.items():
    for artist in artist_ids:
        if artist in artist_labels:
            temp_labels = artist_labels[artist]
            for label in temp_labels:
                label_events[label] = label_events.get(label,[])
                label_events[label].append(event_id)

In [36]:
label_adjectives = {}
for label_id, event_ids in label_events.items():
    for event_id in event_ids:
        label_adjectives[label_id] = label_adjectives.get(label_id,[])
        temp_adjectives = event_adjectives[event_id]
        label_adjectives[label_id].extend(temp_adjectives)

### adjectives + styles

In [44]:
y = ['bleak','quiet','abstract', 'aggressive', 'atmospheric', 'bold', 'brutal', 'cavernous', 'colorful', 'complex', 'conceptual', 'cosmic', 'crisp', 'dark', 'deep', 'dirty', 'elegant', 'emotional', 'energetic', 'esoteric', 'euphoric', 'exotic', 'expansive', 'experimental', 'exquisite', 'filthy', 'frenetic', 'graceful', 'gritty', 'heavy', 'hypnotic', 'industrial', 'insane', 'intelligent', 'intense', 'intricate', 'jazzy', 'lavish', 'manic', 'melodic', 'minimal', 'monstrous', 'odd', 'ominous', 'peculiar', 'percussive', 'playful', 'primal', 'progressive', 'radical', 'raw', 'relentless', 'repetitive', 'rough', 'serene', 'silky', 'smooth', 'solid', 'soulful', 'sparse', 'stylish', 'tribal', 'unnatural', 'unrelenting', 'upbeat', 'vast', 'visceral', 'vocal', 'warm', 'weird', 'wild', 'artistic', 'original', 'gay', 'underground', 'inventive', 'catchy', 'rhythmic', 'organic', 'sensual', 'harsh', 'interesting', 'exuberant', 'subtle', 'slick', 'ethereal', 'drab', 'angelic', 'delicate', 'lively', 'juicy', 'beautiful', 'galactic', 'gorgeous', 'bright', 'tight', 'contemporary', 'fresh', 'rich', 'immense', 'soft']

In [45]:
m = ['classical','delightful','adventurous', 'airy', 'alien', 'amoral', 'anarchic', 'angry', 'archaic', 'arduous', 'artful', 'attractive', 'awesome', 'banal', 'bizarre', 'bouncy', 'breathtaking', 'broody', 'buff', 'buoyant', 'calm', 'ceaseless', 'celestial', 'cerebral', 'cheerful', 'chunky', 'classical', 'cognitive', 'cold', 'compelling', 'cosmical', 'cosy', 'creative', 'crude', 'dainty', 'decadent', 'desirous', 'destructive', 'devilish', 'devoid', 'dimensional', 'diminutive', 'directional', 'dissimilar', 'distinctive', 'dreamlike', 'dreamy', 'dusky', 'earthy', 'eclectic', 'effective', 'emphatic', 'enigmatic', 'enjoyable', 'ephemeral', 'epic', 'eternal', 'ethnic', 'evident', 'evocative', 'explosive', 'extravagant', 'fabulous', 'fast', 'fearless', 'fecund', 'feminine', 'feverish', 'fierce', 'fitful', 'flamboyant', 'floral', 'florid', 'formative', 'forthright', 'free', 'frictional', 'frilly', 'frothy', 'full', 'furious', 'future', 'fuzzy', 'gargantuan', 'garish', 'ghastly', 'glamorous', 'glassy', 'gnarl', 'grand', 'grandiose', 'grisly', 'guttural', 'harmonic', 'hearty', 'heavenly', 'hedonistic', 'hefty', 'hollow', 'honorable', 'idiomatic', 'imaginative', 'imperfect', 'impressionistic', 'incomparable', 'individual', 'integrative', 'interplanetary', 'jerky', 'jovial', 'joyful', 'jubilant', 'kinesthetic', 'leisurely', 'literal', 'loose', 'lustful', 'luxurious', 'magic', 'malicious', 'mammoth', 'maniacal', 'marvelous', 'meager', 'mechanical', 'melodious', 'mental', 'messy', 'mild', 'miniature', 'modest', 'monotonous', 'multidimensional', 'murky', 'musical', 'mysterious', 'nebular', 'nebulous', 'neural', 'neuronal', 'newtonian', 'noisy', 'oblique', 'opalescent', 'operatic', 'oppressive', 'orchestral', 'otherworldly', 'outlying', 'palliative', 'paradigmatic', 'paranoid', 'passionate', 'phenomenal', 'potential', 'powerful', 'prodigal', 'prodigious', 'provocative', 'punishable', 'querulous', 'radiant', 'rampant', 'raucous', 'remorseless', 'restless', 'revelatory', 'riotous', 'rocky', 'roomy', 'sacred', 'sad', 'sadistic', 'salient', 'schizophrenic', 'seductive', 'seismic', 'sensational', 'sexual', 'shaky', 'shallow', 'silly', 'simple', 'sinister', 'sizable', 'smart', 'smug', 'snazzy', 'soft', 'somber', 'sonorous', 'sordid', 'spacious', 'special', 'spectacular', 'spectral', 'speedy', 'spicy', 'spiritual', 'splashy', 'sprightly', 'square', 'straightforward', 'strange', 'strict', 'sudden', 'surprising', 'surrealist', 'sweaty', 'sweet', 'swingy', 'symphonic', 'terrific', 'teutonic', 'tidal', 'tortuous', 'transcendental', 'tropical', 'tuneful', 'turbulent', 'twisty', 'unusual', 'velvety', 'virulent', 'vivacious', 'void', 'wavy', 'willowy', 'wonderful', 'worthy', 'abnormal', 'apocalyptic', 'arboreal', 'ardent', 'arty', 'bestial', 'drizzly', 'dystopian', 'elastic', 'fanciful', 'homogeneous', 'languid', 'liquid', 'magnificent', 'overwhelming', 'rancorous', 'salacious', 'scary', 'staggering', 'subversive', 'sumptuous', 'surly', 'alternative', 'chilly', 'clear', 'fetid', 'happy', 'low', 'obscene', 'perfect', 'spontaneous', 'stark', 'tentative', 'vivid', 'moody', 'nasty', 'lovely', 'astral', 'barbarous', 'collosal', 'enormous', 'massive', 'savage', 'saucy', 'easy', 'monotone', 'transcendent', 'queer', 'dank', 'tranquil', 'wobbly', 'messy', 'ruthless', 'frantic', 'sexy', 'jumpy', 'expressive', 'offbeat', 'blissful', 'thunderous', 'sensuous', 'mellow', 'gloomy', 'hectic', 'choppy', 'ecstatic', 'vibrant', 'erratic', 'infectious', 'luscious', 'peaceful', 'ferocious', 'kinetic', 'edgy', 'unconventional']

In [46]:
ym = y+m
ym = list(set(ym))

In [47]:
styles = ['Abstract','Acid','Acid House','Acid Jazz','Ambient','Ballroom','Baltimore Club','Bassline','Beatdown','Berlin-School','Big Beat','Breakbeat','Breakcore','Breaks','Broken Beat','Chillwave','Chiptune','Dance-pop','Dark Ambient','Darkwave','Deep House'
,'Deep Techno','Disco','Disco Polo','Donk','Doomcore','Downtempo','Drone','Drum n Bass','Dub','Dub Techno','Dubstep','Dungeon Synth','EBM','Electro','Electro House','Electroclash','Euro House','Euro-Disco','Eurobeat','Eurodance','Experimental','Freestyle'
,'Funkot','Future Jazz','Gabber','Garage House','Ghetto','Ghetto House','Ghettotech','Glitch','Goa Trance','Grime','Hands Up','Happy Hardcore','Hard Beat','Hard House','Hard Techno','Hard Trance','Hardcore','Hardstyle','Harsh Noise Wall','Hi NRG'
,'Hip Hop','Hip-House','House','IDM','Illbient','Industrial','Italo House','Italo-Disco','Italodance','J-Core','Jazzdance','Juke','Jumpstyle','Jungle','Latin','Leftfield','Lento Violento','Makina','Minimal','Minimal Techno','Modern Classical','Musique Concrète'
,'Neo Trance','Neofolk','Nerdcore Techno','New Age','New Beat','New Wave','Noise','Nu-Disco','Power Electronics','Progressive Breaks','Progressive House','Progressive Trance','Psy-Trance','Rhythmic Noise','Schranz','Skweee','Sound Collage','Speed Garage','Speedcore'
,'Synth-pop','Synthwave','Tech House','Tech Trance','Techno','Trance','Tribal','Tribal House','Trip Hop','Tropical House','UK Funky','UK Garage','Vaporwave','Witch House']

In [48]:
styles = [style.lower() for style in styles]

In [51]:
uncounted_adjective_list = []
for event_id,adjectives in label_adjectives.items():
    uncounted_adjective_list.extend(adjectives)
counted_adjective_list = dict(Counter(uncounted_adjective_list))

In [52]:
out = []
for label_id,adjectives in label_adjectives.items():
    filtered_adjectives = [adjective for adjective in adjectives if adjective in ym]
    if len(filtered_adjectives) >= 100:
        item_out = {}
        item_out['label_id'] = label_id
        item_out['adjectives'] = filtered_adjectives
        out.append(item_out)

label_adjectives = out

In [54]:
label_ids = [item['label_id'] for item in label_adjectives]

In [55]:
label_adjectives = [item['adjectives'] for item in label_adjectives]

In [85]:
len(label_adjectives)

17741

In [73]:
len(label_ids)

17741

In [56]:
all_adjectives = list(set([item for sublist in label_adjectives for item in sublist]))

In [57]:
len(all_adjectives)

400

In [58]:
counted_adjective_list = {adjective:value for adjective,value in counted_adjective_list.items() if adjective in all_adjectives}

### make labels with adjective ratios

In [235]:
label_adjectives_merged = []
for i in range(len(label_adjectives)):
    label_adjectives_merged.append({
        'label_id': label_ids[i],
        'number_adjectives': len(label_adjectives[i]),
        'label_adjectives_normalised': {},
        'label_adjectives_counted': label_adjectives[i]
    })

In [236]:
for label in label_adjectives_merged:
    counted = Counter(label['label_adjectives_counted'])
    label['label_adjectives_counted'] = counted

In [376]:
label_adjectives_counted = deepcopy(label_adjectives_merged)

In [377]:
label_adjectives_counted[0]

x_min = 1000
x_max = 0

for label in label_adjectives_counted:
    for adjective, count in label['label_adjectives_counted'].items():
        count /= label['number_adjectives']
        count /= counted_adjective_list[adjective]
        label['label_adjectives_normalised'][adjective] = count
        
        if count < x_min:
            x_min = count
        if count > x_max:
            x_max = count
            
new_x_min = 1000
new_x_max = 0
            
for label in label_adjectives_counted:
    label['label_adjectives_normalised_adjectives'] = []
    label['label_adjectives_normalised_counts'] = []
    for adjective, count in label['label_adjectives_normalised'].items():
        count = (count - x_min) / (x_max - x_min)
        label['label_adjectives_normalised_adjectives'].append(adjective)
        label['label_adjectives_normalised_counts'].append(count)
        
        if count < new_x_min:
            new_x_min = count
        if count > new_x_max:
            new_x_max = count

In [378]:
# x_norm = (x - x_min) / (x_max - x_min)

In [379]:
new_x_min

0.0

In [380]:
new_x_max

1.0

In [459]:
label_adjectives_normalised = deepcopy(label_adjectives_counted)

In [460]:
def sort_index(lst, rev=True):
    index = range(len(lst))
    s = sorted(index, reverse=rev, key=lambda i: lst[i])
    return s

In [461]:
for label in label_adjectives_normalised:
    sorted_index = sort_index(label['label_adjectives_normalised_counts'])[:5]
    
    label['adjectives'] = []

    for index in sorted_index:
        label['adjectives'].append(label['label_adjectives_normalised_adjectives'][index])
  
    label.pop('label_adjectives_normalised_counts')
    label.pop('label_adjectives_normalised_adjectives')
    label.pop('label_adjectives_normalised')
    label.pop('label_adjectives_counted')
    label.pop('number_adjectives')

In [462]:
label_adjectives_normalised[0]

{'label_id': '135249-Neutonmusic',
 'adjectives': ['saucy', 'ceaseless', 'erratic', 'monotonous', 'bestial']}

In [463]:
labels = pickle.load( open( "pkl/labels_final.pkl", "rb" ) )

In [464]:
for label in label_adjectives_normalised:
    label_id = label['label_id']
    try:
        labels[label_id]['adjectives'] = label['adjectives']
    except:
        pass
    
out = []
for label in labels.values():
    if 'adjectives' in label.keys():
        out.append(label)

In [472]:
label_dict = {}
for label in out:
    label_id = label['label_id']
    new_out[label_id] = label

In [ ]:
out = []

for record in records.values():
    try:
        label = record['record_labels'][0]
        print(label)
        record['label_adjectives'] = label_dict[label]['adjectives']
        out.append(record)
    except:
        pass

5-Svek
5-Svek
6-Ruffhouse-Records
7-F-111-Records
8-Rather-Interesting
5-Svek
9-Deep4Life
9-Deep4Life
10-Naked-Music-Recordings
10-Naked-Music-Recordings
10-Naked-Music-Recordings
10-Naked-Music-Recordings
9-Deep4Life
2-Earthtones-Recordings
3-Seasons-Recordings
26-Intrinsic-Design
26-Intrinsic-Design
26-Intrinsic-Design
27-Leaf-Recordings
24-Immigrant
18-Soma-Quality-Recordings
9-Deep4Life
9-Deep4Life
586116-Digitalsoul
586116-Digitalsoul
1379296-Soul-Searching-2
1379296-Soul-Searching-2
29-Recline
30-Palette-Recordings
31-Mosaic
1379296-Soul-Searching-2
32-Worship-Recordings
29074-Moonshine-Music
34-Statra-Recordings
35-Ovum-Recordings
36-End-Recordings
37-Zebra-Recordings
39-i!-Records
39-i!-Records
38-Metro-Trax-Records
5-Svek
40-Loaded-Records
40-Loaded-Records
40-Loaded-Records
41-Fathom
25-Mixer
10-Naked-Music-Recordings
40-Loaded-Records
12-Large-Records
28-Deep-Vision-Records
812428-Building-Blocks
45-Groovin-Records
24-Immigrant
1488-Alleviated-Records
60-Fax-49-69450464
61-I

In [481]:
out[0:10]

[]

In [408]:
for label in label_adjectives_normalised:
    if 'quiet' in label['adjectives']:
        print(label)

{'label_id': '25847-Skint', 'adjectives': ['rocky', 'quiet', 'languid', 'directional', 'nasty']}
{'label_id': '702-OM-Records', 'adjectives': ['operatic', 'angelic', 'jubilant', 'quiet', 'buff']}
{'label_id': '609-Strictly-Rhythm', 'adjectives': ['salacious', 'flamboyant', 'sizable', 'virulent', 'quiet']}
{'label_id': '54439-none', 'adjectives': ['quiet', 'ethereal', 'evocative', 'stark', 'insane']}
{'label_id': '35673-Fahrenheit-Music', 'adjectives': ['neural', 'sparse', 'alien', 'savage', 'quiet']}
{'label_id': '48459-Flashover-Recordings', 'adjectives': ['vivacious', 'ferocious', 'euphoric', 'broody', 'quiet']}
{'label_id': '588961-Misch-Masch', 'adjectives': ['quiet', 'unnatural', 'evocative', 'silky', 'mechanical']}
{'label_id': '27314-Fine', 'adjectives': ['quiet', 'unnatural', 'evocative', 'tuneful', 'wavy']}
{'label_id': '10119-P-Vine-Records', 'adjectives': ['quiet', 'nebulous', 'imaginative', 'mechanical', 'diminutive']}
{'label_id': '40812-Dohb-Discs', 'adjectives': ['broody

In [348]:
for label in label_adjectives_normalised:
    new = {}
    for adjective, count in label['label_adjectives_normalised'].items():
        if count > 0.0005:
            new[adjective] = count
    label['label_adjectives_normalised'] = new
    label.pop('label_adjectives_counted')
    label.pop('number_adjectives')

In [468]:
label_adjectives_normalised[100:110]

[{'label_id': '627610-Skint-2',
  'adjectives': ['literal', 'gloomy', 'prodigal', 'raucous', 'terrific']},
 {'label_id': '425883-Warner-Vision-Australia',
  'adjectives': ['gloomy', 'literal', 'prodigal', 'raucous', 'terrific']},
 {'label_id': '32905-Flamingo-Recordings',
  'adjectives': ['grand', 'literal', 'salacious', 'raucous', 'cheerful']},
 {'label_id': '40569-Fat-2',
  'adjectives': ['gargantuan', 'exuberant', 'obscene', 'crude', 'cheerful']},
 {'label_id': '97280-Edelpitch-AB',
  'adjectives': ['literal', 'raucous', 'gloomy', 'prodigal', 'tranquil']},
 {'label_id': '20239-Toolroom-Trax',
  'adjectives': ['choppy',
   'collosal',
   'unrelenting',
   'melodious',
   'luxurious']},
 {'label_id': '134228-Not-On-Label-Cut-The-Funk-Series',
  'adjectives': ['glassy', 'furious', 'revelatory', 'tidal', 'literal']},
 {'label_id': '97778-The-Sun-2',
  'adjectives': ['literal', 'gloomy', 'prodigal', 'raucous', 'terrific']},
 {'label_id': '135084-Not-On-Label-Eminem',
  'adjectives': ['li

# autoencoder

In [59]:
V = len(all_adjectives)
N = len(label_adjectives)

In [62]:
A = np.zeros((V, N))

j = 0
for label in label_adjectives:
    for word in label:
        i = all_adjectives.index(word)
        A[i,j] += 1
    
    if j%1000 == 0:
        print(j)

    j += 1

A = A.T
print(A.shape)

adj_sums = {}
for j in A: # j is label index
    label_sum = 0
    for i in range(len(j)):
        adj_sums[i] = adj_sums.get(i,0)
        adj_sums[i] += j[i]
        

for j in A: # j is label index
    label_sum = 0
    for i in range(len(j)):          
        j[i] /= adj_sums[i]
        label_sum += j[i]
    
    for i in range(len(j)):
        j[i] /= label_sum

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
(17741, 400)
